cart决策树包括cart分类树和cart回归树，基于前面的"1_cart分类树"和"cart回归树"算法可见，
二者处理的不同在于计算方式是gini/variance。因此可以将二者统一成一个算法

**cart分类树**

In [2]:
class Node(object):
    """通过树结点的各属性记录生成的树结构"""
    def __init__(self,
                 best_feature_i=None, 
                 best_split_point=None,
                 left_node=None, 
                 right_node=None,
                 leaf_class = None,
                 is_leaf=False,
                 gini=None):
        """
        每个当前结点Node都记录了当前的划分状况
        :param left_child_node : 结点的左侧子结点
        :param right_child_node : 结点的右侧子结点
        :param best_feature_i : 当前结点的最佳划分特征
        :param split_point : 当前结点的最佳特征对应的最佳分割点
        :param leaf_class : 记录当前节点所属的类别
        :param is_leaf : 只有在is_leaf==True时，leaf_class才生效
        :param gini : 当前节点的gini_index
        
        """
        self.best_feature_i = best_feature_i
        self.best_split_point = best_split_point
        self.left_node = left_node
        self.right_node = right_node
        self.leaf_class = leaf_class
        self.is_leaf = is_leaf
        self.gini = gini
class CartDecisionTree():
    """使用cart算法构建决策树"""
    
    def __init__(self, max_depth = float("inf"),min_sample_split=2, min_gini_decrease=None):
        # 代表决策树的决策树根节点
        self.root_node = None 
        # 预设的决策树最大深度
        self.max_depth = max_depth
        # 预设的决策树叶子节点最小样本数
        self.min_sample_split = min_sample_split
        # 预设的基尼系数增益的最小值（gini_gain太小时不划分）
        self.min_gini_decrease = min_gini_decrease
    def fit(self, X,y,is_linear=False):
        """
        决策树拟合
        :param X : 训练数据集∈（m,n）
        :param y : 训练标签∈（n,1）
        :param is_linear : 特征是否为连续型
        
        """
        # 创建决策树根结点
        self.root_node = Node()
        # 默认根节点的深度为1
        cur_depth = 1
        # 根节点的初始化权重
        # 样本的初始权重:都为1
        weight = np.ones((len(X))) # 全局的weight:初始化为全1 
        # 递归构建决策树
        self._build_tree_recussive(X,y,np.arange(len(X)),weight,self.root_node, cur_depth, is_linear)
    
    def _build_tree_recussive(self, X,y, node_indices,weight,node:Node, cur_depth, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,递归建立决策树
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param weight : 所有样本对应的权重
        :param node : 当前结点的状态记录

        """
        n_samples = len(node_indices)
        n_features = X.shape[1]
        # 记录本节点的状态
        node.gini = self._gini(y[node_indices], weight[node_indices])
        node.leaf_class = self._majority_vote(y[node_indices], weight[node_indices])

        ## 递归基
        # 节点包含数据属于同一个类别，此时无需划分
        if len(np.unique(y[node_indices])) <= 1:
            # 记录叶子结点所属的分类
            node.is_leaf = True
            return
        # 没有更多特征(当前节点所含样本所有特征都只有一个取值)
        if np.sum([len(np.unique(X[node_indices][:,i])) for i in range(n_features)]) == n_features:
            node.is_leaf = True
            return
        # 限制构建子树的深度
        if cur_depth >= self.max_depth:
            node.is_leaf = True
            return
        # 限制节点的最小样本量
        if n_samples < self.min_sample_split:
            node.is_leaf = True
            return

        ## 处理当前节点自身
        # 找到最佳特征和特征的最佳分割点
        best_feature_i,best_gini_gain, best_sets = self._get_best_split_feature(X, y, node_indices, weight, is_linear)
        
        # 基尼系数增益的最小值（gini_gain太小时不划分）
        if self.min_gini_decrease is not None and  best_gini_gain < self.min_gini_decrease:
            node.is_leaf = True
            return
        
        # 基于最佳特征和最佳分割点分成左右子树left,right
        left_indices = best_sets["left_indices"]
        right_indices = best_sets["right_indices"]
        left_weight = best_sets["left_weight"]
        right_weight = best_sets["right_weight"]
        # 记录本节点的状态
        node.best_feature_i = best_feature_i
        node.best_split_point = best_sets["best_split_point"]
        node.left_node = Node()
        node.right_node = Node()
        # --leaf_class和gini在递归基时记录

        # 让buildtree()帮忙划分左右子树
        self._build_tree_recussive(X,y,left_indices,left_weight, node.left_node, cur_depth+1, is_linear)
        self._build_tree_recussive(X,y,right_indices,right_weight, node.right_node, cur_depth+1, is_linear)
                                   
    def _get_best_split_feature(self,X,y,node_indices,weight, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,找到最佳特征：求所有特征的gini_gain
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param weight : 所有样本对应的权重
        :param is_linear : 特征的类型

        """

        # 获取样本数和特征数
        n_samples, n_features = len(node_indices), X.shape[1]
        
        # 初始化
        best_gini_gain = -1
        best_feature = None
        best_sets = None
        
        # 依次求所有特征的gini_gain
        for feature_i in range(n_features):
            # 特征在所有样本中取值唯一时无需找split_point和参与特征划分
            if len(np.unique(X[node_indices][:,feature_i])) == 1:
                continue
                
            ## 找出未缺失的样本
            nonan_indices = [i  for i in node_indices if ~np.isnan(X[i, feature_i])]
            # 找出缺失样本
            nan_indices = [i  for i in node_indices if np.isnan(X[i, feature_i])]
            # 无缺失值样本所占的比例:对每一个样本赋予了权重后,利用权重计算无缺失样本所占的比例
            lou = np.sum(weight[nonan_indices]) / np.sum(weight[node_indices])
            
            # 特征i的基尼增益以及分割的左右子树
            cur_gini_gain,cur_branch_sets = self._get_best_split_point(X,y,nonan_indices, weight,feature_i, is_linear)
            # 找到特征i的最佳gini_point后,使用权重计算最终的gini_gain
            cur_gini_gain = lou * cur_gini_gain
            
            # 寻找最佳特征
            if cur_gini_gain >= best_gini_gain:
                best_gini_gain = cur_gini_gain
                best_feature = feature_i
                best_sets = cur_branch_sets
                # 修改权重
                left_weight, right_weight = np.zeros_like(weight),np.zeros_like(weight)
                left_indices, right_indices = best_sets["left_indices"], best_sets["right_indices"]
                left_weight[left_indices], right_weight[right_indices] = weight[left_indices], weight[right_indices]
                left_weight[nan_indices], right_weight[nan_indices] = np.sum(weight[left_indices]) / np.sum(weight[nonan_indices]),np.sum(weight[right_indices]) / np.sum(weight[nonan_indices])

                # 将缺失样本按不同的比重放到左右两个子集中
                left_indices.extend(nan_indices)
                right_indices.extend(nan_indices)
                best_sets["left_indices"] = left_indices
                best_sets["right_indices"] = right_indices
                best_sets["left_weight"] = left_weight
                best_sets["right_weight"] = right_weight

        # 找到了当前节点所用的最佳特征（也找到了该特征的最佳分割点）
        
        return best_feature,best_gini_gain, best_sets
    
    def _get_best_split_point(self,X,y,node_indices, weight, feature_i, is_linear=False):

        """
        对于当前节点集合（X，y）-node_indices计算特征i的基尼增益
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引(无缺失值)
        :param weight: 所有样本的权重
        :param is_linear : 特征的类型
        :return : 返回特征i的基尼增益以及分割的左右子树

        """
        ## 基于无缺失的样本来寻找特征的最佳切分点
        # 当前特征i的所有候选分割点
        split_points, split_func = self._create_split_points(X[node_indices], feature_i, is_linear)

        ## 产生最佳切分点
        # 初始化
        best_gini_gain = -1# 存储本特征的最佳切分点对应的gini_gain
        best_sets = None# 存储最佳切分点切分的左右分支
        # 依次使用候选分割点对当前集合（X，y）进行二分分割
        for point in split_points:
            # 使用每个候选分割点进行二分分割
            left_indices = [i for i in node_indices if split_func(X[i],point)]
            right_indices = [i for i in node_indices if not split_func(X[i], point)]
            # 分好左右分支后计算划分后的gini_gain
            # 左右分支的权重计算不再使用频数
            w_left, w_right = np.sum(weight[left_indices])/np.sum(weight), np.sum(weight[right_indices])/np.sum(weight)

            # 未划分前的gini:使用该属性上无缺失的样本来计算
            cur_gini = self._gini(y[node_indices], weight[node_indices])
            # 未划分前-划分后==gini_gain
            cur_gini_gain = cur_gini - (w_left*self._gini(y[left_indices],weight[left_indices]) + w_right*self._gini(y[right_indices], weight[right_indices]))

            # 选择最佳的split point
            if cur_gini_gain >= best_gini_gain:
                best_gini_gain = cur_gini_gain
                # 划分时传给左右子集的weight、indices均不同weigh
                # weight在之后再重置
                best_sets = {
                    "best_split_point":point,
                    "left_indices": left_indices,
                    "right_indices": right_indices,
                }
        return best_gini_gain, best_sets
    
    def _create_split_points(self, X, feature_i, is_linear=False):
    
        """
        根据特征i是连续型/离散型特征得到特征i的所有候选分割点,并返回对应的分割函数
        :param X: 当前集合的样本（无缺失值）
        :param feature_i : 给定的特征索引
        :return 特征i的所有候选分割点、分割函数

        """

        # 1、确定特征i的所有可能取值
        feature_values = np.unique(X[:, feature_i])# 已排序的 

        # 2、根据特征i是连续型/离散型特征对这些可能取值进行处理从而得到特征i的所有候选分割点；    
        split_points = None
        split_func = None

        if is_linear:
            split_points = (feature_values[1:] + feature_values[:-1]) / 2 # 特征是连续型特征则使用二分法找到所有的切分点
            split_func = lambda x,split_point : x[feature_i] >= split_point

        else:
            split_points = feature_values# 离散型特征直接使用特征的各个取值作为切分点
            split_func = lambda x,split_point : x[feature_i] == split_point
        return split_points, split_func
    
     
    
    def _gini(self,y, weight):
        
        """
        计算当前样本集合的gini指数
        :param y : 当前集合对应的标签
        :param weight : 当前样本集合对应的权重

        """
        # 找出y中包含的所有类别
        unique_y = np.unique(y)
        weight_sum = np.sum(weight)
        # 1.计算p_k:计算无缺失样本中第k类所占的比例
        p_k = [np.sum(weight[y==k])/ weight_sum for k in unique_y ]
        # 2.计算gini
        gini = np.sum([p * (1-p) for p in p_k])
        return gini
    
    def _majority_vote(self, y,weight):
        """
        根据多数原则定叶子节点所属的分类
        :param y : 当前集合对应的标签
        :param weight : 前集合对应的权重
        :return most_common_class叶子节点所属的分类
        """
        most_common_class = None
        max_distribution = 0
        for k in np.unique(y): 
            distribution = np.sum(weight[y==k])
            if distribution >= max_distribution:
                max_distribution = distribution
                most_common_class = k
        return most_common_class
    
    def prune(self, alpha=0):
        """对决策树进行后剪枝"""
        return self._pruning_node(self.root_node, alpha)
        
    def _pruning_node(self, node,alpha):
        """
        :param node : 当前处理的节点
        :param alpha: loss的参数，alpha≥0

        """
        ## 递归基:当前节点是叶子节点则直接返回
        if node.is_leaf:
            return 
        ## 让递归函数帮忙处理左右子树
        self._pruning_node(node.left_node, alpha)
        self._pruning_node(node.right_node, alpha)

        ## 处理当前节点
        # 剪枝后
        post_loss = node.gini + alpha * 1 
        # 剪枝前
        pre_loss = node.left_node.gini + node.right_node.gini + alpha * 2
        # 比较剪枝前的loss与剪枝后的loss
        if post_loss < pre_loss: # 剪枝后loss更小则剪枝（收回左右结点）
            node.left_node = None
            node.right_node = None
            node.best_feature_i = None
            node.best_split_point = None
            node.is_leaf = True
    
    def predict(self,X,is_linear=False):
        """
        :param X: 待预测的m个样本

        """
        # 每一个样本都通过二叉搜索决策树树查找所属类别,决策树由其根节点作为代表
        y_pred = [self._search_class(x, self.root_node, is_linear) for x in X]
        return y_pred

    def _search_class(self, x, node:Node, is_linear=False):
        """
        : param x: 待预测所属分类的样本
        : param node : 当前所在节点

        """
        # 递归基
        if node.is_leaf:# 已经走到叶子
            return node.leaf_class
        ## 当前节点的工作
        # 本样本最终要往哪个分支走
        goto = None
        # 根据当前节点的最佳特征及最佳切分点决定x是继续往左边走还是往右边走
        feature_value = x[node.best_feature_i]
        # 离散型/连续型特征处理不同
        if is_linear:
            if feature_value >= node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node#往右边
        else:
            if feature_value == node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node
        return self._search_class(x, goto, is_linear)
        
    


**cart回归树**

In [3]:
class Node(object):
    """通过树结点的各属性记录生成的树结构"""
    def __init__(self,
                 best_feature_i=None, 
                 best_split_point=None,
                 left_node=None, 
                 right_node=None,
                 leaf_value = None,
                 is_leaf=False,
                 variance=None):
        """
        每个当前结点Node都记录了当前的划分状况
        :param left_child_node : 结点的左侧子结点
        :param right_child_node : 结点的右侧子结点
        :param best_feature_i : 当前结点的最佳划分特征
        :param split_point : 当前结点的最佳特征对应的最佳分割点
        :param leaf_value : 记录当前节点的预测值
        :param is_leaf : 只有在is_leaf==True时，leaf_class才生效
        :param variance : 当前节点的平方误差
        
        """
        self.best_feature_i = best_feature_i
        self.best_split_point = best_split_point
        self.left_node = left_node
        self.right_node = right_node
        self.leaf_value = leaf_value
        self.is_leaf = is_leaf
        self.variance = variance
        
class CartRegressionTree():
    """使用cart算法构建决策树"""
    
    def __init__(self, max_depth = float("inf"),min_sample_split=2, min_variance_decrease=None):
        # 代表决策树的决策树根节点
        self.root_node = None 
        # 预设的决策树最大深度
        self.max_depth = max_depth
        # 预设的决策树叶子节点最小样本数
        self.min_sample_split = min_sample_split
        # 预设的基尼系数增益的最小值（variance_gain太小时不划分）
        self.min_variance_decrease = min_variance_decrease
        
    def fit(self, X,y,is_linear=False):
        """
        决策树拟合
        :param X : 训练数据集∈（m,n）
        :param y : 训练标签∈（n,1）
        :param is_linear : 特征是否为连续型
        
        """
        # 创建决策树根结点
        self.root_node = Node()
        # 默认根节点的深度为1
        cur_depth = 1
        # 根节点的初始化权重
        # 样本的初始权重:都为1
        weight = np.ones((len(X))) # 全局的weight:初始化为全1 
        # 递归构建决策树
        self._build_tree_recussive(X,y,np.arange(len(X)),weight,self.root_node, cur_depth, is_linear)
    
    def _build_tree_recussive(self, X,y, node_indices,weight,node:Node, cur_depth, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,递归建立决策树
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param weight : 所有样本对应的权重
        :param node : 当前结点的状态记录

        """
        n_samples = len(node_indices)
        n_features = X.shape[1]
        # 记录本节点的状态
        node.variance = self._variance(y[node_indices], weight[node_indices])
        node.leaf_value = self._leaf_value(y[node_indices])

        ## 递归基
        # 节点包含数据全为同一个值，此时无需划分
        if len(np.unique(y[node_indices])) <= 1:
            # 记录叶子结点所属的分类
            node.is_leaf = True
            return
        # 没有更多特征(当前节点所含样本所有特征都只有一个取值)
        if np.sum([len(np.unique(X[node_indices][:,i])) for i in range(n_features)]) == n_features:
            node.is_leaf = True
            return
        # 限制构建子树的深度
        if cur_depth >= self.max_depth:
            node.is_leaf = True
            return
        # 限制节点的最小样本量
        if n_samples < self.min_sample_split:
            node.is_leaf = True
            return

        ## 处理当前节点自身
        # 找到最佳特征和特征的最佳分割点
        best_feature_i,best_variance_gain, best_sets = self._get_best_split_feature(X, y, node_indices, weight, is_linear)
        
        # 基尼系数增益的最小值（gini_gain太小时不划分）
        if self.min_variance_decrease is not None and  best_variance_gain < self.min_variance_decrease:
            node.is_leaf = True
            return
        
        # 基于最佳特征和最佳分割点分成左右子树left,right
        left_indices = best_sets["left_indices"]
        right_indices = best_sets["right_indices"]
        left_weight = best_sets["left_weight"]
        right_weight = best_sets["right_weight"]
        # 记录本节点的状态
        node.best_feature_i = best_feature_i
        node.best_split_point = best_sets["best_split_point"]
        node.left_node = Node()
        node.right_node = Node()
        # --leaf_class和gini在递归基时记录

        # 让buildtree()帮忙划分左右子树
        self._build_tree_recussive(X,y,left_indices,left_weight, node.left_node, cur_depth+1, is_linear)
        self._build_tree_recussive(X,y,right_indices,right_weight, node.right_node, cur_depth+1, is_linear)
                                   
    def _get_best_split_feature(self, X,y,node_indices,weight, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,找到最佳特征：求所有特征的gini_gain
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param is_linear : 特征的类型

        """

        # 获取样本数和特征数
        n_samples, n_features = len(node_indices), X.shape[1]
        
        # 初始化
        best_variance_gain = -1
        best_feature = None
        best_sets = None
        
        # 依次求所有特征的gini_gain
        for feature_i in range(n_features):
            # 特征在所有样本中取值唯一时无需找split_point和参与特征划分
            if len(np.unique(X[node_indices][:,feature_i])) == 1:
                continue
                
            ## 找出未缺失的样本
            nonan_indices = [i  for i in node_indices if ~np.isnan(X[i, feature_i])]
            # 找出缺失样本
            nan_indices = [i  for i in node_indices if np.isnan(X[i, feature_i])]
            # 无缺失值样本所占的比例:对每一个样本赋予了权重后,利用权重计算无缺失样本所占的比例
            lou = np.sum(weight[nonan_indices]) / np.sum(weight[node_indices])
            
            # 特征i的基尼增益以及分割的左右子树
            cur_variance_gain,cur_branch_sets = self._get_best_split_point(X,y,nonan_indices, weight,feature_i, is_linear)
            # 找到特征i的最佳gini_point后,使用权重计算最终的gini_gain
            cur_variance_gain = lou * cur_variance_gain
            
            # 寻找最佳特征
            if cur_variance_gain >= best_variance_gain:
                best_variance_gain = cur_variance_gain
                best_feature = feature_i
                best_sets = cur_branch_sets
                # 修改权重
                left_weight, right_weight = np.zeros_like(weight),np.zeros_like(weight)
                left_indices, right_indices = best_sets["left_indices"],best_sets["right_indices"]
                left_weight[left_indices], right_weight[right_indices] = weight[left_indices], weight[right_indices]
                left_weight[nan_indices], right_weight[nan_indices] = np.sum(weight[left_indices]) / np.sum(weight[nonan_indices]),np.sum(weight[right_indices]) / np.sum(weight[nonan_indices])

                # 将缺失样本按不同的比重放到左右两个子集中
                left_indices.extend(nan_indices)
                right_indices.extend(nan_indices)
                best_sets["left_indices"] = left_indices
                best_sets["right_indices"] = right_indices
                best_sets["left_weight"] = left_weight
                best_sets["right_weight"] = right_weight

        # 找到了当前节点所用的最佳特征（也找到了该特征的最佳分割点）
        
        return best_feature,best_variance_gain, best_sets
    
    def _get_best_split_point(self, X,y,node_indices, weight, feature_i, is_linear=False):

        """
        对于当前节点集合（X，y）-node_indices计算特征i的基尼增益
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引(无缺失值)
        :param weight: 所有样本的权重
        :param is_linear : 特征的类型
        :return : 返回特征i的基尼增益以及分割的左右子树

        """
        ## 基于无缺失的样本来寻找特征的最佳切分点
        # 当前特征i的所有候选分割点
        split_points, split_func = self._create_split_points(X[node_indices], feature_i, is_linear)

        ## 产生最佳切分点
        # 初始化
        best_variance_gain = -1# 存储本特征的最佳切分点对应的gini_gain
        best_sets = None# 存储最佳切分点切分的左右分支
        # 依次使用候选分割点对当前集合（X，y）进行二分分割
        for point in split_points:
            # 使用每个候选分割点进行二分分割
            left_indices = [i for i in node_indices if split_func(X[i],point)]
            right_indices = [i for i in node_indices if not split_func(X[i], point)]
            ## 分好左右分支后计算划分后的variance_gain
            # 左右分支的权重计算不再使用频数
            w_left, w_right = np.sum(weight[left_indices])/np.sum(weight), np.sum(weight[right_indices])/np.sum(weight)

            # 未划分前的平方误差和:使用该属性上无缺失的样本来计算
            cur_variance = self._variance(y[node_indices],weight[node_indices])
            # 未划分前-划分后(左右)==variance_gain
            cur_variance_gain = cur_variance - (w_left*self._variance(y[left_indices],weight[left_indices]) + w_right*self._variance(y[right_indices], weight[right_indices]))

            # 选择最佳的split point
            if cur_variance_gain >= best_variance_gain:
                best_variance_gain = cur_variance_gain
                # 划分时传给左右子集的weight、indices均不同weigh
                # weight在之后再重置
                best_sets = {
                    "best_split_point":point,
                    "left_indices": left_indices,
                    "right_indices": right_indices,
                }
        return best_variance_gain, best_sets
    
    def _create_split_points(self, X, feature_i, is_linear=False):
    
        """
        根据特征i是连续型/离散型特征得到特征i的所有候选分割点,并返回对应的分割函数
        :param X: 当前集合的样本（无缺失值）
        :param feature_i : 给定的特征索引
        :return 特征i的所有候选分割点、分割函数

        """

        # 1、确定特征i的所有可能取值
        feature_values = np.unique(X[:, feature_i])# 已排序的 

        # 2、根据特征i是连续型/离散型特征对这些可能取值进行处理从而得到特征i的所有候选分割点；    
        split_points = None
        split_func = None

        if is_linear:
            split_points = (feature_values[1:] + feature_values[:-1]) / 2 # 特征是连续型特征则使用二分法找到所有的切分点
            split_func = lambda x,split_point : x[feature_i] >= split_point

        else:
            split_points = feature_values# 离散型特征直接使用特征的各个取值作为切分点
            split_func = lambda x,split_point : x[feature_i] == split_point
        return split_points, split_func
    
     
    
    def _variance(self, y, weight):
        """
        :param y : 当前节点样本的连续标签
        :param weight : 当前节点样本的权重
        :return square_variance : 平方误差和

        """
        mean_y = np.mean(y, axis=0)
        sum_square_variance = np.sum(np.square(y-mean_y)*weight)
        return sum_square_variance

    def _leaf_value(self, y):
        return np.mean(y, axis=0)
    
   
    
    def predict(self,X,is_linear=False):
        """
        :param X: 待预测的m个样本

        """
        # 每一个样本都通过二叉搜索决策树树查找所属类别,决策树由其根节点作为代表
        y_pred = [self._search_yhat(x, self.root_node, is_linear) for x in X]
        return y_pred

    def _search_yhat(self, x, node:Node, is_linear=False):
        """
        : param x: 待预测所属分类的样本
        : param node : 当前所在节点

        """
        # 递归基
        if node.is_leaf:# 已经走到叶子
            return node.leaf_value
        ## 当前节点的工作
        # 本样本最终要往哪个分支走
        goto = None
        # 根据当前节点的最佳特征及最佳切分点决定x是继续往左边走还是往右边走
        feature_value = x[node.best_feature_i]
        # 离散型/连续型特征处理不同
        if is_linear:
            if feature_value >= node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node#往右边
        else:
            if feature_value == node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node
        return self._search_yhat(x, goto, is_linear)
        
    def prune(self, alpha=0):
        """对决策树进行后剪枝"""
        return self._pruning_node(self.root_node, alpha)
        
    def _pruning_node(self, node,alpha):
        """
        :param node : 当前处理的节点
        :param alpha: loss的参数，alpha≥0

        """
        ## 递归基:当前节点是叶子节点则直接返回
        if node.is_leaf:
            return 
        ## 让递归函数帮忙处理左右子树
        self._pruning_node(node.left_node, alpha)
        self._pruning_node(node.right_node, alpha)

        ## 处理当前节点
        # 剪枝后
        post_loss = node.variance + alpha * 1 
        # 剪枝前
        pre_loss = node.left_node.variance + node.right_node.variance + alpha * 2
        # 比较剪枝前的loss与剪枝后的loss
        if post_loss < pre_loss: # 剪枝后loss更小则剪枝（收回左右结点）
            node.left_node = None
            node.right_node = None
            node.best_feature_i = None
            node.best_split_point = None
            node.is_leaf = True


**汇合成cart决策树**

先定义共同的部分

In [1]:
class Node(object):
    """通过树结点的各属性记录生成的树结构"""
    def __init__(self,
                 best_feature_i=None, 
                 best_split_point=None,
                 left_node=None, 
                 right_node=None,
                 leaf_value = None,
                 is_leaf=False,
                 metric=None):
        """
        每个当前结点Node都记录了当前的划分状况
        :param left_child_node : 结点的左侧子结点
        :param right_child_node : 结点的右侧子结点
        :param best_feature_i : 当前结点的最佳划分特征
        :param split_point : 当前结点的最佳特征对应的最佳分割点
        :param leaf_value : 记录当前节点所属的类别(分类树)or均值(回归树)
        :param is_leaf : 只有在is_leaf==True时，leaf_class才生效
        :param metric : 当前节点的gini_index(分类树)or平方误差variance(回归树)
        
        """
        self.best_feature_i = best_feature_i
        self.best_split_point = best_split_point
        self.left_node = left_node
        self.right_node = right_node
        self.leaf_value = leaf_value
        self.is_leaf = is_leaf
        self.metric = metric
        
class CartDecisionTree():
    """使用cart算法构建决策树框架"""
    
    def __init__(self, max_depth = float("inf"),min_sample_split=2, min_metric_decrease=None):
        # 代表决策树的决策树根节点
        self.root_node = None 
        ## 以下是预剪枝相关参数
        # 预设的决策树最大深度
        self.max_depth = max_depth
        # 预设的决策树叶子节点最小样本数
        self.min_sample_split = min_sample_split
        # 预设的基尼系数增益的最小值（variance_gain太小时不划分）
        self.min_metric_decrease = min_metric_decrease
        ## 以下是分类/回归树的特定参数
        # 计算gini/variance的函数
        self.compute_metric = None
        # 计算叶子节点类别/预测均值的函数
        self.compute_leaf_value = None
        
    def fit(self, X,y,is_linear):
        """
        决策树拟合
        :param X : 训练数据集∈（m,n）
        :param y : 训练标签∈（n,1）
        :param is_linear : 特征是否为连续型
        
        """
        # 创建决策树根结点
        self.root_node = Node()
        # 默认根节点的深度为1
        cur_depth = 1
        # 根节点的初始化权重
        # 样本的初始权重:都为1
        weight = np.ones((len(X))) # 全局的weight:初始化为全1 
        # 递归构建决策树
        self._build_tree_recussive(X,y,np.arange(len(X)),weight,self.root_node, cur_depth, is_linear)
    
    def _build_tree_recussive(self, X,y, node_indices,weight,node:Node, cur_depth, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,递归建立决策树
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param weight : 所有样本对应的权重
        :param node : 当前结点的状态记录

        """
        n_samples = len(node_indices)
        n_features = X.shape[1]
        # 记录本节点的状态
        node.metric = self.compute_metric(y[node_indices], weight[node_indices])
        node.leaf_value = self.compute_leaf_value(y[node_indices], weight[node_indices])

        ## 递归基
        # 节点包含数据全为同一个值，此时无需划分
        if len(np.unique(y[node_indices])) <= 1:
            # 记录叶子结点所属的分类
            node.is_leaf = True
            return
        # 没有更多特征(当前节点所含样本所有特征都只有一个取值)
        if np.sum([len(np.unique(X[node_indices][:,i])) for i in range(n_features)]) == n_features:
            node.is_leaf = True
            return
        # 限制构建子树的深度
        if cur_depth >= self.max_depth:
            node.is_leaf = True
            return
        # 限制节点的最小样本量
        if n_samples < self.min_sample_split:
            node.is_leaf = True
            return

        ## 处理当前节点自身
        # 找到最佳特征和特征的最佳分割点
        best_feature_i,best_gain, best_sets = self._get_best_split_feature(X, y, node_indices, weight, is_linear)
        
        # 增益的最小值（gini_gain太小时不划分）
        if self.min_metric_decrease is not None and  best_gain < self.min_metric_decrease:
            node.is_leaf = True
            return
        
        # 基于最佳特征和最佳分割点分成左右子树left,right
        left_indices = best_sets["left_indices"]
        right_indices = best_sets["right_indices"]
        left_weight = best_sets["left_weight"]
        right_weight = best_sets["right_weight"]
        # 记录本节点的状态
        node.best_feature_i = best_feature_i
        node.best_split_point = best_sets["best_split_point"]
        node.left_node = Node()
        node.right_node = Node()
        # --leaf_class和gini在递归基时记录

        # 让buildtree()帮忙划分左右子树
        self._build_tree_recussive(X,y,left_indices,left_weight, node.left_node, cur_depth+1, is_linear)
        self._build_tree_recussive(X,y,right_indices,right_weight, node.right_node, cur_depth+1, is_linear)
                                   
    def _get_best_split_feature(self, X,y,node_indices,weight, is_linear):
        """
        对于当前节点集合（X，y）-node_indices,找到最佳特征：求所有特征的gini_gain
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引
        :param is_linear : 特征的类型

        """

        # 获取样本数和特征数
        n_samples, n_features = len(node_indices), X.shape[1]
        
        # 初始化
        best_gain = -1
        best_feature = None
        best_sets = None
        
        # 依次求所有特征的gini_gain
        for feature_i in range(n_features):
            # 特征在所有样本中取值唯一时无需找split_point和参与特征划分
            if len(np.unique(X[node_indices][:,feature_i])) == 1:
                continue
                
            ## 找出未缺失的样本
            nonan_indices = [i  for i in node_indices if ~np.isnan(X[i, feature_i])]
            # 找出缺失样本
            nan_indices = [i  for i in node_indices if np.isnan(X[i, feature_i])]
            # 无缺失值样本所占的比例:对每一个样本赋予了权重后,利用权重计算无缺失样本所占的比例
            lou = np.sum(weight[nonan_indices]) / np.sum(weight[node_indices])
            
            # 特征i的增益以及分割的左右子树
            cur_gain,cur_branch_sets = self._get_best_split_point(X,y,nonan_indices, weight,feature_i, is_linear)
            # 找到特征i的最佳gini_point后,使用权重计算最终的gini_gain
            cur_gain = lou * cur_gain
            
            # 寻找最佳特征
            if cur_gain >= best_gain:
                best_gain = cur_gain
                best_feature = feature_i
                best_sets = cur_branch_sets
                # 修改权重
                left_weight, right_weight = np.zeros_like(weight),np.zeros_like(weight)
                left_indices, right_indices = best_sets["left_indices"],best_sets["right_indices"]
                left_weight[left_indices], right_weight[right_indices] = weight[left_indices], weight[right_indices]
                left_weight[nan_indices], right_weight[nan_indices] = np.sum(weight[left_indices]) / np.sum(weight[nonan_indices]),np.sum(weight[right_indices]) / np.sum(weight[nonan_indices])

                # 将缺失样本按不同的比重放到左右两个子集中
                left_indices.extend(nan_indices)
                right_indices.extend(nan_indices)
                best_sets["left_indices"] = left_indices
                best_sets["right_indices"] = right_indices
                best_sets["left_weight"] = left_weight
                best_sets["right_weight"] = right_weight

        # 找到了当前节点所用的最佳特征（也找到了该特征的最佳分割点）
        
        return best_feature,best_gain, best_sets
    
    def _get_best_split_point(self, X,y,node_indices, weight, feature_i, is_linear):

        """
        对于当前节点集合（X，y）-node_indices计算特征i的基尼增益
        :param X: 所有样本
        :param y: 所有标签
        :param node_indices : 当前样本集合对应的索引(无缺失值)
        :param weight: 所有样本的权重
        :param is_linear : 特征的类型
        :return : 返回特征i的基尼增益以及分割的左右子树

        """
        ## 基于无缺失的样本来寻找特征的最佳切分点
        # 当前特征i的所有候选分割点
        split_points, split_func = self._create_split_points(X[node_indices], feature_i, is_linear)

        ## 产生最佳切分点
        # 初始化
        best_gain = -1# 存储本特征的最佳切分点对应的gini_gain
        best_sets = None# 存储最佳切分点切分的左右分支
        # 依次使用候选分割点对当前集合（X，y）进行二分分割
        for point in split_points:
            # 使用每个候选分割点进行二分分割
            left_indices = [i for i in node_indices if split_func(X[i],point)]
            right_indices = [i for i in node_indices if not split_func(X[i], point)]
            ## 分好左右分支后计算划分后的variance_gain
            # 左右分支的权重计算不再使用频数
            w_left, w_right = np.sum(weight[left_indices])/np.sum(weight), np.sum(weight[right_indices])/np.sum(weight)

            # 未划分前的平方误差和/gini:使用该属性上无缺失的样本来计算
            cur_metric = self.compute_metric(y[node_indices],weight[node_indices])
            # 未划分前-划分后(左右)==variance_gain
            cur_gain = cur_metric - (w_left*self.compute_metric(y[left_indices],weight[left_indices]) + w_right*self.compute_metric(y[right_indices], weight[right_indices]))

            # 选择最佳的split point
            if cur_gain >= best_gain:
                best_gain = cur_gain
                # 划分时传给左右子集的weight、indices均不同weigh
                # weight在之后再重置
                best_sets = {
                    "best_split_point":point,
                    "left_indices": left_indices,
                    "right_indices": right_indices,
                }
        return best_gain, best_sets
    
    def _create_split_points(self, X, feature_i, is_linear):
    
        """
        根据特征i是连续型/离散型特征得到特征i的所有候选分割点,并返回对应的分割函数
        :param X: 当前集合的样本（无缺失值）
        :param feature_i : 给定的特征索引
        :return 特征i的所有候选分割点、分割函数

        """

        # 1、确定特征i的所有可能取值
        feature_values = np.unique(X[:, feature_i])# 已排序的 

        # 2、根据特征i是连续型/离散型特征对这些可能取值进行处理从而得到特征i的所有候选分割点；    
        split_points = None
        split_func = None

        if is_linear:
            split_points = (feature_values[1:] + feature_values[:-1]) / 2 # 特征是连续型特征则使用二分法找到所有的切分点
            split_func = lambda x,split_point : x[feature_i] >= split_point

        else:
            split_points = feature_values# 离散型特征直接使用特征的各个取值作为切分点
            split_func = lambda x,split_point : x[feature_i] == split_point
        return split_points, split_func
    
    def predict(self,X,is_linear=False):
        """
        :param X: 待预测的m个样本

        """
        # 每一个样本都通过二叉搜索决策树树查找所属类别,决策树由其根节点作为代表
        y_pred = [self._search_yhat(x, self.root_node, is_linear) for x in X]
        return y_pred

    def _search_yhat(self, x, node:Node, is_linear):
        """
        : param x: 待预测所属分类的样本
        : param node : 当前所在节点

        """
        # 递归基
        if node.is_leaf:# 已经走到叶子
            return node.leaf_value
        ## 当前节点的工作
        # 本样本最终要往哪个分支走
        goto = None
        # 根据当前节点的最佳特征及最佳切分点决定x是继续往左边走还是往右边走
        feature_value = x[node.best_feature_i]
        # 离散型/连续型特征处理不同
        if is_linear:
            if feature_value >= node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node#往右边
        else:
            if feature_value == node.best_split_point:
                goto = node.left_node# 往左边
            else:
                goto = node.right_node
        return self._search_yhat(x, goto, is_linear)
        
    def prune(self, alpha=0):
        """对决策树进行后剪枝"""
        return self._pruning_node(self.root_node, alpha)
        
    def _pruning_node(self, node,alpha):
        """
        :param node : 当前处理的节点
        :param alpha: loss的参数，alpha≥0

        """
        ## 递归基:当前节点是叶子节点则直接返回
        if node.is_leaf:
            return 
        ## 让递归函数帮忙处理左右子树
        self._pruning_node(node.left_node, alpha)
        self._pruning_node(node.right_node, alpha)

        ## 处理当前节点
        # 剪枝后
        post_loss = node.metric + alpha * 1 
        # 剪枝前
        pre_loss = node.left_node.metric + node.right_node.metric + alpha * 2
        # 比较剪枝前的loss与剪枝后的loss
        if post_loss < pre_loss: # 剪枝后loss更小则剪枝（收回左右结点）
            node.left_node = None
            node.right_node = None
            node.best_feature_i = None
            node.best_split_point = None
            node.is_leaf = True


**定义分类树的特殊部分**

In [2]:
class CartClassificationTree(CartDecisionTree):
    """定义Cart分类树"""
    def _gini(self,y, weight):
        
        """
        计算当前样本集合的gini指数
        :param y : 当前集合对应的标签
        :param weight : 当前样本集合对应的权重

        """
        # 找出y中包含的所有类别
        unique_y = np.unique(y)
        weight_sum = np.sum(weight)
        # 1.计算p_k:计算无缺失样本中第k类所占的比例
        p_k = [np.sum(weight[y==k])/ weight_sum for k in unique_y ]
        # 2.计算gini
        gini = np.sum([p * (1-p) for p in p_k])
        return gini
    
    def _majority_vote(self, y,weight):
        """
        根据多数原则定叶子节点所属的分类
        :param y : 当前集合对应的标签
        :param weight : 前集合对应的权重
        :return most_common_class叶子节点所属的分类
        """
        most_common_class = None
        max_distribution = 0
        for k in np.unique(y): 
            distribution = np.sum(weight[y==k])
            if distribution >= max_distribution:
                max_distribution = distribution
                most_common_class = k
        return most_common_class 
    
    def fit(self, X,y, is_linear=False):
        """分类树拟合"""
        # 为之前的CartDecision进行初始化
        self.compute_metric = self._gini
        self.compute_leaf_value = self._majority_vote
        super(CartClassificationTree, self).fit(X,y,is_linear)

In [13]:
class CartRegressionTree(CartDecisionTree):
    """定义cart回归树"""
    def _variance(self, y, weight):
        """
        :param y : 当前节点样本的连续标签
        :param weight : 当前节点样本的权重
        :return square_variance : 平方误差和

        """
        mean_y = np.mean(y, axis=0)
        sum_square_variance = np.sum(np.square(y-mean_y)*weight)
        return sum_square_variance

    def _leaf_value(self, y,weight):
        return np.mean(y, axis=0)
    
    def fit(self, X, y, is_linear=False):
        """回归树拟合"""
        # 为之前的CartDecision进行初始化
        self.compute_metric = self._variance
        self.compute_leaf_value = self._leaf_value
        super(CartRegressionTree, self).fit(X,y,is_linear)

分类树测试

In [4]:
import numpy as np

In [5]:
def accuracy(y_pred, y_true):
    return np.sum(y_pred == y_true) / len(y_pred)

In [6]:
X = np.array([
       [np.nan, 1, 1, 1, 1, 1],
       [1, 1, 2, 1, 1, np.nan],
       [1, 1, np.nan, 1, 1, 1],
       [2, 1, 2, 1, 1, 1],
       [np.nan, 1, 1, 1, 1, 1],
       [2, 2, 1, 1, np.nan, 2],
       [1, 2, 1, 2, 2, 2],
       [1, 2, 1, np.nan, 2, 1],
       [1, np.nan, 2, 2, 2, 1],
       [2, 3, 3, np.nan, 3, 2],
       [3, 3, 3, 3, 3, np.nan],
       [3, 1, np.nan, 3, 3, 2],
       [np.nan, 2, 1, 2, 1, 1],
       [3, 2, 2, 2, 1, 1],
       [1, 2, 1, 1, np.nan, 2],
       [3, 1, 1, 3, 3, 1],
       [2, np.nan, 2, 2, 2, 1]])
y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
model = CartClassificationTree()
model.fit(X,y)
y_pred = model.predict(X)
accuracy(y_pred, y)

1.0

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
data = load_iris()
X,y = data.data, data.target
X_train,test_X,y_train,test_y = train_test_split(X,y,test_size=0.3)


In [8]:
# 不输入参数——不进行预剪枝
model = CartClassificationTree()
model.fit(X_train, y_train, is_linear=True)
y_pred = model.predict(test_X, is_linear=True)

accuracy(y_pred, test_y)

1.0

In [9]:
# 输入参数——预剪枝
model = CartClassificationTree(max_depth=3, min_sample_split=3, min_metric_decrease=0.1)
model.fit(X_train,y_train, is_linear=True)
y_pred = model.predict(test_X, is_linear=True)
accuracy(y_pred,test_y)

0.9555555555555556

In [10]:
model.prune(0.2)
y_pred = model.predict(test_X)
accuracy(y_pred,test_y)

0.28888888888888886

回归树测试

In [11]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
X, y = load_boston(return_X_y=True)
X_train, test_X, y_train, test_y = train_test_split(X,y, test_size=0.3)


D:\yangdaopy\Pycharm\pythondownload\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housin

In [14]:
# 不使用参数预剪枝
from sklearn.metrics import accuracy_score, mean_squared_error
model = CartRegressionTree()
model.fit(X_train, y_train, is_linear=True)
y_pred = model.predict(test_X, is_linear=True)
mean_squared_error(test_y, y_pred)

14.842105263157896

In [16]:
# 使用参数预剪枝
from sklearn.metrics import accuracy_score, mean_squared_error
model = CartRegressionTree(max_depth = 6,min_sample_split=5, min_metric_decrease=3)
model.fit(X_train, y_train, is_linear=True)
y_pred = model.predict(test_X, is_linear=True)
mean_squared_error(test_y, y_pred)

13.207565599995371

In [25]:
# 创建决策树不使用参数预剪枝
from sklearn.metrics import accuracy_score, mean_squared_error
model = CartRegressionTree()
model.fit(X_train, y_train, is_linear=True)
y_pred = model.predict(test_X, is_linear=True)
print(mean_squared_error(test_y, y_pred))
# 创建决策树后进行后剪枝
model.prune(0.999)
y_pred = model.predict(test_X, is_linear=True)
print(mean_squared_error(test_y, y_pred))

14.842105263157896
14.682051892006804
